In [1]:
from ipyleaflet import Map, Marker
from lib.client import Api
import pandas as pd

client = Api()

# this function allows to map a value from a domain of min-max to another
def map_value(value, domain_min, domain_max, range_min, range_max):
    left_span = domain_max - domain_min
    right_span = range_max - range_min

    # Convert the left range into a 0-1 range (float)
    scaled = float(value - domain_min) / float(left_span)

    # Convert the 0-1 range into a value in the right range.
    return range_min + (scaled * right_span)

# doc : https://ipyleaflet.readthedocs.io/en/latest/api_reference/circle_marker.html

In [2]:
travels = client.portic.get_flows({"date": "1789"})

In [3]:
center = (45.6876849, -1.15)
poitou_map = Map(center=center,zoom=5)

for travel in travels[0:10]:
    marker = Marker(location=(travel["departure_latitude"],travel["departure_longitude"]))
    poitou_map.add_layer(marker)
          
display(poitou_map)

Map(center=[45.6876849, -1.15], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [4]:
ports = {}

for travel in travels:
    if travel["departure_uhgs_id"] in ports:
        ports[travel["departure_uhgs_id"]]["count"] += 1
    else:
        ports[travel["departure_uhgs_id"]] = {
             "count": 1,
             'departure_fr': travel['departure_fr'],
             'departure_en': travel['departure_en'],
             'latitude': travel['departure_latitude'],
             'longitude': travel['departure_longitude'],
             "id": travel["departure_uhgs_id"]
        }
ports = [port for key,port in ports.items()]

from ipyleaflet import basemaps, basemap_to_tiles, CircleMarker

watercolor = basemap_to_tiles(basemaps.Stamen.Watercolor)

center = (45.6876849, -1.15)

poitou_map = Map(layers=(watercolor, ), center=center, zoom=5)

max_count = max([port["count"] for port in ports])
min_count = min([port["count"] for port in ports])
MIN_RADIUS = 1
MAX_RADIUS = 20

for port in ports:
    circle_marker = CircleMarker()
    circle_marker.location = (port["latitude"], port["longitude"])
    circle_marker.radius = int(map_value(port["count"], min_count, max_count, MIN_RADIUS, MAX_RADIUS))
    circle_marker.color = "red"
    circle_marker.fill_color = "red"
    poitou_map.add_layer(circle_marker)

poitou_map

Map(center=[45.6876849, -1.15], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…